![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [73]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
# Import any additional modules and start coding below

In [74]:
data = pd.read_csv('rental_info.csv')
data.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401


In [75]:
# Data Pre Processing

In [76]:
# Adding the rental_length_days column
data['rental_length_days'] = (pd.to_datetime(data['return_date']) - pd.to_datetime(data['rental_date'])).dt.days
data.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,rental_length_days
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,3
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,7
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,4


In [77]:
data['deleted_scenes'] = np.where(data['special_features'].str.contains('Deleted Scenes'), 1, 0)
data['behind_the_scenes'] = np.where(data['special_features'].str.contains('Behind the Scenes'), 1, 0)
data.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,rental_length_days,deleted_scenes,behind_the_scenes
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,3,0,1
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2,0,1
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,7,0,1
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2,0,1
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,4,0,1


In [78]:
# for the improvement of results. The project doesn't require this, hence, the omition for now.

#data['rental_weekday'] = pd.to_datetime(data['rental_date']).dt.weekday
#data['rental_month'] = pd.to_datetime(data['rental_date']).dt.month
#data['is_weekend'] = (data['rental_weekday'] >= 5).astype(int)

In [79]:
cols_to_drop = ['special_features', 'rental_length_days', 'rental_date', 'return_date']

X = data.drop(cols_to_drop, axis = 1)
y = data['rental_length_days']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

In [80]:
# Lasso
lasso = Lasso(alpha = 0.3, random_state = 9)
lasso.fit(X_train, y_train)
lasso_coef = lasso.coef_

selected_features = X_train.columns[lasso_coef != 0]
X_lasso_train, X_lasso_test = X_train[selected_features], X_test[selected_features]


ols = LinearRegression()
ols = ols.fit(X_lasso_train, y_train)
y_test_pred = ols.predict(X_lasso_test)
mse_lin_reg_lasso = mean_squared_error(y_test, y_test_pred)

print(" MSE for linear reg lasso: ", mse_lin_reg_lasso)

 MSE for linear reg lasso:  3.0923563956298517


In [81]:
# Random Forest
rf = RandomForestRegressor()
param = {
    'n_estimators': np.arange(50, 300, 50),  # Increase range
    'max_depth': np.arange(5, 21, 5),       # Allow deeper trees
    'min_samples_split': [2, 5, 10],        # Control tree branching
    'min_samples_leaf': [1, 2, 4],          # Prevent overfitting
    'max_features': ['sqrt', 'log2'],       # Try different splits
}


random_search = RandomizedSearchCV(rf, param_distributions=param, cv = 5, random_state = 9)

random_search.fit(X_train, y_train)
hyper_params = random_search.best_params_

rf = RandomForestRegressor(n_estimators=hyper_params['n_estimators'],
                          max_depth=hyper_params['max_depth'],
                          random_state = 9)

rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
mse_random_forest = mean_squared_error(y_test, rf_pred)

print(" MSE for random forest: ", mse_random_forest)

 MSE for random forest:  2.0264091991606445


In [82]:
# XGBoost
xgb = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=9)
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)

mse_xgb = mean_squared_error(y_test, xgb_pred)
print("MSE for XGBoost:", mse_xgb)


MSE for XGBoost: 2.2004465016641386


In [83]:
test_results = X_test.copy()
test_results['Actual_Rental_Days'] = y_test
test_results['Predicted_Rental_Days'] = np.round(rf_pred).astype(int)

print(test_results[['Actual_Rental_Days', 'Predicted_Rental_Days']].head(10))  # View first 10 predictions


       Actual_Rental_Days  Predicted_Rental_Days
15067                   8                      7
3808                    1                      3
1015                    6                      2
12617                   9                      8
1711                    5                      3
508                     3                      2
3008                    1                      5
12810                   0                      2
4776                    6                      7
10275                   7                      8


In [84]:
# Best model
best_model = rf
best_mse = mse_random_forest
